In [ ]:
import numpy as np
import pandas as pd
from itertools import product
import os
from Bio import SeqIO

In [13]:
nucleotides = ['A', 'C', 'G', 'T']
combinations_of_3 = [''.join(combination) for combination in product(nucleotides, repeat=3)]
kolone = ['ID', 'Tip']
kolone.extend(combinations_of_3)
kolone.extend(['Pocetak', 'Kraj'])
kolone

['ID',
 'Tip',
 'AAA',
 'AAC',
 'AAG',
 'AAT',
 'ACA',
 'ACC',
 'ACG',
 'ACT',
 'AGA',
 'AGC',
 'AGG',
 'AGT',
 'ATA',
 'ATC',
 'ATG',
 'ATT',
 'CAA',
 'CAC',
 'CAG',
 'CAT',
 'CCA',
 'CCC',
 'CCG',
 'CCT',
 'CGA',
 'CGC',
 'CGG',
 'CGT',
 'CTA',
 'CTC',
 'CTG',
 'CTT',
 'GAA',
 'GAC',
 'GAG',
 'GAT',
 'GCA',
 'GCC',
 'GCG',
 'GCT',
 'GGA',
 'GGC',
 'GGG',
 'GGT',
 'GTA',
 'GTC',
 'GTG',
 'GTT',
 'TAA',
 'TAC',
 'TAG',
 'TAT',
 'TCA',
 'TCC',
 'TCG',
 'TCT',
 'TGA',
 'TGC',
 'TGG',
 'TGT',
 'TTA',
 'TTC',
 'TTG',
 'TTT',
 'Pocetak',
 'Kraj']

In [3]:
# genomi sa nevalidnim slovima
genoms = ['NC_008563','NC_004431','NC_002655','NC_017632','NC_017641']

In [22]:
genoms = ['NC_017626', 'NC_017627', 'NC_013364', 'NC_013365', 'NC_013366', 'NC_013361', 'NC_013369', 
          'NC_013353', 'NC_013354', 'NC_018650', 'NC_018654', 'NC_018661', 'NC_018662', 'NC_018658', 
          'NC_018659', 'NC_018666', 'NC_008253', 'NC_011748', 'NC_017631', 'NC_008563', 'NC_009837', 
          'NC_009838', 'NC_020163', 'NC_010468', 'NC_012892', 'NC_012971', 'NC_013941', 'NC_017646', 
          'NC_004431', 'NC_017625', 'NC_017638', 'NC_011601', 'NC_009786', 'NC_009788', 'NC_009790', 
          'NC_009801', 'NC_011353', 'NC_011745', 'NC_002655', 'NC_007414', 'NC_017633', 'NC_012947', 
          'NC_009800', 'NC_011741', 'NC_011750', 'NC_017628', 'NC_007779', 'NC_010473', 'NC_012759', 
          'NC_020518', 'NC_000913', 'NC_016902', 'NC_016904', 'NC_017660', 'NC_011993', 'NC_022364', 
          'NC_017644', 'NC_017634', 'NC_017659', 'NC_017663', 'NC_022370', 'NC_012967', 'NC_017656', 
          'NC_017657', 'NC_011742', 'NC_011747', 'NC_011415', 'NC_011419', 'NC_013654', 'NC_013655', 
          'NC_010488', 'NC_010498', 'NC_002128', 'NC_002695', 'NC_013008', 'NC_013010', 'NC_017630', 
          'NC_017632', 'NC_011739', 'NC_011749', 'NC_011751', 'NC_017639', 'NC_017641', 'NC_017642', 
          'NC_017645', 'NC_007941', 'NC_007946', 'NC_017635', 'NC_017637', 'NC_017664', 'NC_017665', 
          'NC_017906', 'NC_017907', 'NC_017652', 'NC_017651']

In [14]:
len(genoms)

95

# Brojanje kodona za PAI ostrva

In [5]:
for genom in genoms:

    fasta_file = f"{genom}/{genom}.fasta"
    sequences = SeqIO.parse(fasta_file, "fasta")

    if os.path.exists(f"{genom}/patogena_ostrva.csv"):
        pai_ostrva = pd.read_csv(f"{genom}/patogena_ostrva.csv")

        niska = None
        for record in sequences:
            niska = record.seq

        for index, row in pai_ostrva.iterrows():
            i = row['Start']
            while row['End'] != i + 2:
                kodon = niska[i:i+3]
                if kodon in pai_ostrva.columns:
                    pai_ostrva.at[index, kodon] += 1
                else:
                    print(f"Kodon {kodon} nije validan.")
                    break
                i += 1
        pai_ostrva.to_csv(f"{genom}/patogena_ostrva.csv", index=False)
        print(f"Zavrsen {genom}.")
    else:
        print("Nema PAI ostrva.")


# Dodavanje PAI ostrva u konacnu tabelu

In [19]:
nucleotides = ['A', 'C', 'G', 'T']
combinations_of_3 = list(product(nucleotides, repeat=3))

kolone1 = [''.join(combination) for combination in combinations_of_3]

In [23]:
konacna_tabela = pd.DataFrame(columns=kolone)
for genom in genoms:
        if os.path.exists(f"{genom}/patogena_ostrva.csv"):
                pai_ostrva = pd.read_csv(f"{genom}/patogena_ostrva.csv")
                for index, row in pai_ostrva.iterrows():
                        novi_red = {'ID': genom}
                        konacna_tabela = konacna_tabela.append(novi_red, ignore_index=True)
                        novi_red_indeks = konacna_tabela.index[-1]
                        if row['Type'] == 0:
                                konacna_tabela.at[novi_red_indeks,"Tip"] = "P" 
                        elif row['Type'] == 1:
                                 konacna_tabela.at[novi_red_indeks,"Tip"] = "C"
                        else:
                                konacna_tabela.at[novi_red_indeks,"Tip"] = "N"

                        konacna_tabela.at[novi_red_indeks,"Pocetak"] = pai_ostrva.at[index,"Start"]
                        konacna_tabela.at[novi_red_indeks,"Kraj"] = pai_ostrva.at[index,"End"]

                        for kodon in kolone1:
                                if kodon in konacna_tabela.columns:
                                        konacna_tabela.at[novi_red_indeks,kodon] = pai_ostrva.at[index,kodon]
                                else:
                                        print(f"Kodon {kodon} nije validan.")
                print(f"Zavrsen {genom}.")
        else:
                print("Nema PAI ostrva.")
konacna_tabela

Zavrsen NC_017626.
Zavrsen NC_017627.
Zavrsen NC_013364.
Zavrsen NC_013365.
Zavrsen NC_013366.
Zavrsen NC_013361.
Zavrsen NC_013369.
Zavrsen NC_013353.
Zavrsen NC_013354.
Zavrsen NC_018650.
Zavrsen NC_018654.
Zavrsen NC_018661.
Zavrsen NC_018662.
Zavrsen NC_018658.
Zavrsen NC_018659.
Zavrsen NC_018666.
Zavrsen NC_008253.
Zavrsen NC_011748.
Zavrsen NC_017631.
Zavrsen NC_008563.
Zavrsen NC_009837.
Zavrsen NC_009838.
Zavrsen NC_020163.
Zavrsen NC_010468.
Zavrsen NC_012892.
Zavrsen NC_012971.
Zavrsen NC_013941.
Zavrsen NC_017646.
Zavrsen NC_004431.
Zavrsen NC_017625.
Zavrsen NC_017638.
Zavrsen NC_011601.
Nema PAI ostrva.
Nema PAI ostrva.
Zavrsen NC_009790.
Zavrsen NC_009801.
Zavrsen NC_011353.
Zavrsen NC_011745.
Zavrsen NC_002655.
Zavrsen NC_007414.
Zavrsen NC_017633.
Zavrsen NC_012947.
Zavrsen NC_009800.
Zavrsen NC_011741.
Zavrsen NC_011750.
Zavrsen NC_017628.
Zavrsen NC_007779.
Zavrsen NC_010473.
Zavrsen NC_012759.
Zavrsen NC_020518.
Zavrsen NC_000913.
Zavrsen NC_016902.
Zavrsen NC_01690

,ID,Tip,AAA,AAC,AAG,AAT,ACA,ACC,ACG,ACT,...,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT,Pocetak,Kraj
0,NC_017626,C,991,597,601,748,566,499,563,429,...,784,802,786,656,589,798,658,1009,233779,275861
1,NC_017626,C,1455,1371,807,1185,1196,1234,1191,851,...,1371,1490,1273,860,929,1201,969,1395,2272389,2345178
2,NC_017626,C,1058,555,443,988,462,301,214,424,...,511,380,278,417,762,656,517,1178,3249573,3277026
3,NC_017626,C,589,393,311,435,401,335,257,295,...,432,402,315,334,331,465,287,489,3387125,3408657
4,NC_017626,C,1577,1192,804,1389,917,1006,997,727,...,1153,1273,935,861,1173,1314,993,1894,3429556,3495247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,NC_017651,C,2531,1934,1346,2046,1667,1706,1502,1223,...,1935,2097,1534,1515,1737,2103,1530,2874,3316685,3423733
569,NC_017651,C,679,420,341,523,310,289,278,293,...,451,360,327,281,437,422,353,628,4145358,4166721
570,NC_017651,C,513,395,253,321,365,379,302,275,...,514,370,433,274,294,307,210,318,4903949,4924373
571,NC_017651,N,197,138,111,198,104,136,156,86,...,170,192,179,121,122,142,168,200,1420583,1430066


In [24]:
konacna_tabela.to_csv(f"konacna_tabela.csv", index=False)